In [ ]:
import pandas as pd; 
import numpy as np
import csv
import glob
from pandas.plotting import lag_plot
from kmodes.kmodes import KModes
from kmodes.util.dissim import ng_dissim


In [ ]:
act = pd.read_csv('') ## add .csv filename of complete data 

permit = perm.drop(columns = ['Unnamed: 0'])
act = act.drop(columns = 'Unnamed: 0')
permit = act[act['class'] == "p"]
df = pd.DataFrame(permit)


In [ ]:
# Clustring using KMODES

data =  df #FN.drop(columns=['class','lable']) #FP.drop(columns=['class','lable'])#ReFN.drop(columns=['class','lable','lable1','lable2']) #ReFN.drop(columns=['class','lable','lable1']).reset_index() #FN.drop(columns=['class','lable']) #df
#data = data.drop(columns=['index'])

k =  10 #,25,35]
datasize= len(data.index)
print(datasize)
km = KModes(n_clusters = k, init='Cao', n_init=1, verbose=2)
    # Huang, Cao

df_dummy = pd.get_dummies(data) ## FN ,FP , df

clusters = km.fit_predict(data)
df_dummy['clusters'] = clusters

# Print the cluster centroids

centroids=km.cluster_centroids_
    
#print(centroids)

# Mappling the data to clusters

cluster_map = pd.DataFrame()
cluster_map['data_index'] = data.index.values
#print(cluster_map.head(3))
cluster_map['cluster'] = km.labels_
    
#valuescount=pd.DataFrame()
#temp=[]

for y in range(0, k):
        ind = cluster_map.cluster == y
        #print(ind)

ndf=pd.concat([data,cluster_map], axis=1)
ndf=ndf.drop(columns='data_index')
#print(cluster_map)


    # Size of each cluster
ndf.groupby('cluster').count()

print(ndf.describe())

In [ ]:
# Calculating Attributes Effectevness in each cluster
import time
start_time = time.time()

temp = pd.DataFrame()
#print(datafreq)
clusTemp = pd.DataFrame()

policy ={}
###change FN: p, FP: d or act
datasize= len(act.index)
#print("{")
for i in range(0,k):
    policy[i] = {}
    
    clusTemp = ndf.loc[ndf['cluster'] == i]
    #clusTemp = FN.loc[FN['cluster'] == i]
    clussize= len(clusTemp.index)
    #print(i,":{",sep='',end="")
    #print(clusTemp.describe())
    for col in df.columns:
        #actual data:  act[col].value_counts().to_dict()
        # FN data: p[col].value_counts().to_dict()
        # FP data: d[col].value_counts().to_dict()
        datafreq =   act[col].value_counts().to_dict() # act[col].value_counts().to_dict()#
        
        #datafreq= pd.DataFrame.from_dict(datafreq, )
        temp=clusTemp[col].value_counts().to_dict()
        
        for key, value in datafreq.items():
            
            
            for key1, value1 in temp.items():
                #print(key, key1)
                if key == key1:
                    #print(key ,value,value1)
                    x = value1/datasize
                    y = value1/clussize
                    #print((y-x))
                    #if key in centroids[i]:
                    if (y-x) >= 0.65:
                        if col not in policy[i]:
                            policy[i][col] = key
                    for col2 in df.columns:
                                if col !=col2:
                                    clusTemp = ndf.loc[ndf['cluster'] == i]
                
                                    clus = clusTemp.loc[clusTemp[col] == clusTemp[col2]]
                                    if len(clus) >= len(clusTemp)/2:
                                        #print("'",col,"':'",col2,"',",sep='',end="")
                                        if col not in policy[i] and col2 not in policy[i]: 
                                            policy[i][col] = col2
                                        else:
                                            if col in policy[i]: 
                                                policy[i][col] = col2
                                            if col2 in policy[i] :
                                                del policy[i][col2]
print("--- %s seconds ---" % (time.time() - start_time))    
print(policy)

In [ ]:
## clean extracted policy by findsimilar rules and clean them
from itertools import chain
from collections import defaultdict
rules = policy

def jaccard_similarity(list1, list2):
    intersection = (set(list1.values()).intersection(list2.values()))
    #print(list(set(list1).intersection(list2)))
    union = defaultdict(list)
    for k, v in chain(list1.items(), list2.items()):
        #if v not in intersection:
            union[k].append(v)
    #print(union)    
    
    intersectionl = len(intersection)
    unionl = len(union)
    score = float(intersectionl / unionl)
    if score > 0.7:
                #print(i,j)
                print(key, ",",key2," score =", score,unionl,intersectionl)
                print( (set(list1).intersection(list2)),"\n" ,union)
                #print(intersectionl)
    return float(intersectionl / unionl)

for key, value in policy.items():
    
    for key2, v  in rules.items():
    #print(key, value)
        
        if key !=key2:
            
            #print(key, key2)
            """for z, j in value.items():
                for y, i in v.items():
                     if z == y:"""
            #score = jaccard_similarity_score(value, v)
            score = jaccard_similarity(value, v)
                        #print(i,j)
            """if score > 0.6:
                #print(i,j)
                print(key, ",",k," score =", score)
    """
print('length of the policy is:' sum(len(v) for v in policy.values()))

In [ ]:
# run the extracted policy over the complete data
import time
def dict_compare(d1, d2):
    d1_keys = set(d1.keys())
    d2_keys = set(d2.keys())

    intersect_keys = d1_keys.intersection(d2_keys)
 
    relation = []
    added = d1_keys - d2_keys
    removed = d2_keys - d1_keys
  
    same = set(o for o in intersect_keys if  d1[o] in d2[o] )
   
    if len(same) == len(intersect_keys):
        return added, removed, same, relation
    for key,  o in d2.items():
        if type(o) == str:
                
 
            temp = set(i for i in intersect_keys if type(d2[i]) == str and d2[i] in d1_keys and d1[i] == d1[d2[i]] )
                #print(temp)
            if len(temp)>0:
                    relation = temp
        
           
    return added, removed, same, relation


def ruleCheck (row):
    #print(row)
    lable = 'd'
    drow = row.to_dict()
    
    for key,value in policy.items():
        #print(key)
        added, removed, same, relation = dict_compare(drow, value)
        
        if len(same)+len(relation) >= len(value) or len(same) == len(value):
            #print(len(same)+len(relation), len(value))
            lable = "p"
            row ['class'] ='p'
            return lable

    return lable
start_time = time.time()

act['lable'] = act.apply(ruleCheck, axis=1)
print("--- %s seconds ---" % (time.time() - start_time))    


In [ ]:
#Evaluation
log = act[(act['class'] == "p") & (act['lable'] == "p")]
log2 = act[(act['class'] == "d") & (act['lable'] == "d")]
FN = act[(act['class'] == "p") & (act['lable'] == "d")]
FP = act[(act['class'] == "d") & (act['lable'] == "p")]
p = act[act['class'] == "p"]
d = act[act['class'] == "d"]
#[(df['column_name'] >= A) & (df['column_name'] <= B)]
#print(len(log),'\n', log.head(10))
#print(FN.describe(),FP.describe(),log2.describe(),d.describe())
#log.describe()
print("FN=", len(FN), ", FP=", len(FP), ", TP=", len(log), ", TN=", len(log2))
recall = (len(log)/(len(log)+len(FN)))*100
precesion = (len(log)/(len(log)+len(FP)))*100
accu = ((len(log)+len(log2))/len(act))*100
f = 2*((recall*precesion)/(recall+precesion))
print("recall =",recall, "\nprecession =",precesion, "\naccuracy = ", accu,"\nf-score = ", f)

In [ ]:
# Calculating FN Attributes Effectevness in each cluster

start_time = time.time()

temp = pd.DataFrame()
#print(datafreq)
clusTemp = pd.DataFrame()
FNpolicy = {}

###change FN: p, FP: d or act
datasize= len(p.index)
#print("{")
for i in range(0,k):
    FNpolicy[i] = {}
    
    clusTemp = ndf.loc[ndf['cluster'] == i]
    #clusTemp = FN.loc[FN['cluster'] == i]
    clussize= len(clusTemp.index)
    #print(i,":{",sep='',end="")
    #print(clusTemp.describe())
    for col in df.columns:
        #actual data:  act[col].value_counts().to_dict()
        # FN data: p[col].value_counts().to_dict()
        # FP data: d[col].value_counts().to_dict()
        datafreq =   p[col].value_counts().to_dict() # act[col].value_counts().to_dict()#
        
        #datafreq= pd.DataFrame.from_dict(datafreq, )
        temp=clusTemp[col].value_counts().to_dict()
        
        for key, value in datafreq.items():
            
            
            for key1, value1 in temp.items():
                #print(key, key1)
                if key == key1:
                    #print(key ,value,value1)
                    x = value1/datasize
                    y = value1/clussize
                    #print((y-x))
                    #if key in centroids[i]:
                    if (y-x) >= 0.05:
                        if col not in FNpolicy[i]:
                            FNpolicy[i][col] = key
                            for col2 in df.columns:
                                if col !=col2:
                                    clusTemp = ndf.loc[ndf['cluster'] == i]
                
                                    clus = clusTemp.loc[clusTemp[col] == clusTemp[col2]]
                                    if len(clus) >= len(clusTemp)/2:
                                        #print("'",col,"':'",col2,"',",sep='',end="")
                                        if col not in FNpolicy[i] and col2 not in FNpolicy[i] :
                                            FNpolicy[i][col] = col2
                                       
                                        if col2 in FNpolicy[i]:
                                            del FNpolicy[i][col2]
print("--- %s seconds ---" % (time.time() - start_time))    
print(FNpolicy)
print('length of the policy is:' sum(len(v) for v in policy.values()))

In [ ]:
# refine the extracted policy based on FN policy 

def jaccard_similarity(list1, list2):
        intersection = (set(list1.values()).intersection(list2.values()))
        #print(list(set(list1).intersection(list2)))
        union = defaultdict(list)
        for k, v in chain(list1.items(), list2.items()):
        #if v not in intersection:
            union[k].append(v)
    #print(union)    
    
        intersectionl = len(intersection)
        unionl = len(union)
        score = float(intersectionl / unionl)
        #print(len(policy[key]),len(FNpolicy[key2]))
        if score >= 0.5:
            if len(policy[key]) > len(FNpolicy[key2]) and len(FNpolicy[key2])>=2 :
                #print(len(policy[key]),len(FNpolicy[key2]))
                policy[key] = FNpolicy[key2]
                print(key, ",",key2," score =", score,unionl,intersectionl)
                print( (set(list1).intersection(list2)),"\n" ,union)
                #print(intersectionl)
        
        return float(intersectionl / unionl)
        return float(intersectionl / unionl)
    
#rules = policy
for key, value in policy.items():
    
    for key2, v  in FNpolicy.items():
        #print(key, key2)
        
        #if key !=key2:
            
            #print(key, key2)
            """for z, j in value.items():
                for y, i in v.items():
                     if z == y:"""
            #score = jaccard_similarity_score(value, v)
            score = jaccard_similarity(value, v)
                        #print(i,j)
            """if score > 0.6:
                #print(i,j)
                print(key, ",",k," score =", score)
    """
sum(len(v) for v in policy.values())

In [ ]:
#Evaluation
log = act[(act['class'] == "p") & (act['lable'] == "p")]
log2 = act[(act['class'] == "d") & (act['lable'] == "d")]
FN = act[(act['class'] == "p") & (act['lable'] == "d")]
FP = act[(act['class'] == "d") & (act['lable'] == "p")]
p = act[act['class'] == "p"]
d = act[act['class'] == "d"]
#[(df['column_name'] >= A) & (df['column_name'] <= B)]
#print(len(log),'\n', log.head(10))
#print(FN.describe(),FP.describe(),log2.describe(),d.describe())
#log.describe()
print("FN=", len(FN), ", FP=", len(FP), ", TP=", len(log), ", TN=", len(log2))
recall = (len(log)/(len(log)+len(FN)))*100
precesion = (len(log)/(len(log)+len(FP)))*100
accu = ((len(log)+len(log2))/len(act))*100
f = 2*((recall*precesion)/(recall+precesion))
print("recall =",recall, "\nprecession =",precesion, "\naccuracy = ", accu,"\nf-score = ", f)

In [ ]:
# Calculating FP Attributes Effectevness in each cluster

start_time = time.time()

temp = pd.DataFrame()
#print(datafreq)
clusTemp = pd.DataFrame()
FPpolicy = {}

###change FN: p, FP: d or act
datasize= len(d.index)
#print("{")
for i in range(0,k):
    FPpolicy[i] = {}
    
    clusTemp = ndf.loc[ndf['cluster'] == i]
    #clusTemp = FN.loc[FN['cluster'] == i]
    clussize= len(clusTemp.index)
    #print(i,":{",sep='',end="")
    #print(clusTemp.describe())
    for col in df.columns:
        #actual data:  act[col].value_counts().to_dict()
        # FN data: p[col].value_counts().to_dict()
        # FP data: d[col].value_counts().to_dict()
        datafreq =   d[col].value_counts().to_dict() # act[col].value_counts().to_dict()#
        
        #datafreq= pd.DataFrame.from_dict(datafreq, )
        temp=clusTemp[col].value_counts().to_dict()
        
        for key, value in datafreq.items():
            
            
            for key1, value1 in temp.items():
                #print(key, key1)
                if key == key1:
                    #print(key ,value,value1)
                    x = value1/datasize
                    y = value1/clussize
                    #print((y-x))
                    #if key in centroids[i]:
                    if (y-x) >= 0.05:
                        if col not in FPpolicy[i]:
                            FPpolicy[i][col] = key
                            for col2 in df.columns:
                                if col !=col2:
                                    clusTemp = ndf.loc[ndf['cluster'] == i]
                
                                    clus = clusTemp.loc[clusTemp[col] == clusTemp[col2]]
                                    if len(clus) >= len(clusTemp)/2:
                                        #print("'",col,"':'",col2,"',",sep='',end="")
                                        if col not in FPpolicy[i] and col2 not in FPpolicy[i] :
                                            FPpolicy[i][col] = col2
                                       
                                        if col2 in FPpolicy[i]:
                                            del FPpolicy[i][col2]
print("--- %s seconds ---" % (time.time() - start_time))    
print(FPpolicy)
print('length of the policy is:' sum(len(v) for v in policy.values()))


In [ ]:
# refine the extracted policy based on FN policy 

def jaccard_similarity(list1, list2):
        intersection = (set(list1.values()).intersection(list2.values()))
        #print(list(set(list1).intersection(list2)))
        union = defaultdict(list)
        for k, v in chain(list1.items(), list2.items()):
        #if v not in intersection:
            union[k].append(v)
    #print(union)    
    
        intersectionl = len(intersection)
        unionl = len(union)
        score = float(intersectionl / unionl)
        #print(len(policy[key]),len(FNpolicy[key2]))
        if score >= 0.5:
            if len(policy[key]) < len(FPpolicy[key2])  :
                #print(len(policy[key]),len(FPpolicy[key2]))
                
                policy[key] = FPpolicy[key2]
                print(key, ",",key2," score =", score,unionl,intersectionl)
                print( (set(list1).intersection(list2)),"\n" ,union)
                #print(intersectionl)
        
        return float(intersectionl / unionl)
        return float(intersectionl / unionl)
    
#rules = policy
for key, value in policy.items():
    
    for key2, v  in FPpolicy.items():
        #print(key, key2)
        
        #if key !=key2:
            
            #print(key, key2)
            """for z, j in value.items():
                for y, i in v.items():
                     if z == y:"""
            #score = jaccard_similarity_score(value, v)
            score = jaccard_similarity(value, v)
                        #print(i,j)
            """if score > 0.6:
                #print(i,j)
                print(key, ",",k," score =", score)
    """
sum(len(v) for v in policy.values())

In [ ]:
#Evaluation
log = act[(act['class'] == "p") & (act['lable'] == "p")]
log2 = act[(act['class'] == "d") & (act['lable'] == "d")]
FN = act[(act['class'] == "p") & (act['lable'] == "d")]
FP = act[(act['class'] == "d") & (act['lable'] == "p")]
p = act[act['class'] == "p"]
d = act[act['class'] == "d"]
#[(df['column_name'] >= A) & (df['column_name'] <= B)]
#print(len(log),'\n', log.head(10))
#print(FN.describe(),FP.describe(),log2.describe(),d.describe())
#log.describe()
print("FN=", len(FN), ", FP=", len(FP), ", TP=", len(log), ", TN=", len(log2))
recall = (len(log)/(len(log)+len(FN)))*100
precesion = (len(log)/(len(log)+len(FP)))*100
accu = ((len(log)+len(log2))/len(act))*100
f = 2*((recall*precesion)/(recall+precesion))
print("recall =",recall, "\nprecession =",precesion, "\naccuracy = ", accu,"\nf-score = ", f)